In [1]:
import sys

sys.path.insert(0, "../..")

In [2]:
from src.data.datasets import get_arabic
from src.settings import presets


In [3]:
presets

Settings(datadir=PosixPath('/workspaces/MLopdracht/data/raw'), testurl=HttpUrl('https://archive.ics.uci.edu/ml/machine-learning-databases/00195/Test_Arabic_Digit.txt', ), trainurl=HttpUrl('https://archive.ics.uci.edu/ml/machine-learning-databases/00195/Train_Arabic_Digit.txt', ), testfile=PosixPath('ArabicTest.txt'), trainfile=PosixPath('ArabicTrain.txt'), modeldir=PosixPath('/workspaces/MLopdracht/models'), logdir=PosixPath('/workspaces/MLopdracht/logs'), modelname='model.pt', batchsize=64)

In [45]:
trainstreamer, teststreamer = get_arabic(presets)

2023-06-02 16:09:47.282 | INFO     | src.data_tools:get_file:61 - File /workspaces/MLopdracht/data/raw/ArabicTrain.txt already exists, skip download
2023-06-02 16:09:47.285 | INFO     | src.data_tools:get_file:61 - File /workspaces/MLopdracht/data/raw/ArabicTest.txt already exists, skip download
2023-06-02 16:09:47.286 | INFO     | src.datasets:get_arabic:33 - Loading data from /workspaces/MLopdracht/data/raw/ArabicTrain.txt and /workspaces/MLopdracht/data/raw/ArabicTest.txt
2023-06-02 16:09:47.796 | INFO     | src.datasets:get_arabic:49 - Returning trainstreamer, teststreamer


In [46]:
x, y = next(iter(trainstreamer.stream()))
x.shape, y.shape

(torch.Size([64, 55, 13]), torch.Size([64]))

In [48]:
from src.models import metrics
accuracy = metrics.Accuracy()


In [52]:
from src.settings import TrainerSettings
from src.settings import TrainerSettings


ImportError: cannot import name 'TrainerSettings' from 'src.settings' (/workspaces/MLopdracht/dev/notebooks/../../src/settings.py)

In [55]:
from src.models import rnn_models
from src.settings import TrainerSettings

import torch
from src.models import train_model

settings = TrainerSettings(
    epochs=10,
    metrics=[accuracy],
    logdir="modellog",
    train_steps=len(trainstreamer),
    valid_steps=len(teststreamer),
    tunewrite=["tensorboard", "gin"]
    )
trainer = train_model.Trainer(
    model=rnn_models.GRUmodel,
    settings=settings,
    loss_fn=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=teststreamer,
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau
    )
trainer.loop()


ImportError: cannot import name 'rnn_models' from 'src.models' (/workspaces/MLopdracht/dev/notebooks/../../src/models.py)

In [21]:
import mlflow
from datetime import datetime
import torch
from src.models import train_model
from pathlib import Path
import gin

from src.models import rnn_models

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("arabic")
modeldir = Path("../../models/arabic/").resolve()
# gin.parse_config_file("gestures.gin")
gin.parse_config_file("test.gin")

config = {
    "input_size":3,
    "hidden_size":128,
    "dropout":0,
    "num_layers":2,
    "output_size":20
}

with mlflow.start_run():

    # mlflow.log_params(gin.get_bindings("BaseRNN"))
    gin.bind_parameter("GRUmodel.config", config)
    mlflow.log_params(gin.get_bindings("GRUmodel"))
    mlflow.log_params(gin.get_bindings("trainloop"))
    mlflow.log_param("datadir", f"{data_dir}")
    mlflow.log_param("batchsize", f"{batchsize}")

    model = rnn_models.GRUmodel()

    model, testloss = train_model.trainloop(
        # loss_fn=torch.nn.CrossEntropyLoss(),
        # optimizer=torch.optim.Adam,
        # epochs=10,
        # learning_rate=1e-3,
        model=model,
        metrics=[accuracy],
        train_dataloader=trainstreamer,
        test_dataloader=teststreamer,
        log_dir="modellog",
        train_steps=len(trainstreamer),
        eval_steps=len(teststreamer),
        tunewriter=["tensorboard", "mlflow", "gin"]
    )
    mlflow.pytorch.log_model(model, "gru")

    # tag = datetime.now().strftime("%Y%m%d-%H%M")
    # modelpath = modeldir / (tag + "model.pt")
    # torch.save(model, modelpath)
    # mlflow.log_artifact(local_path=modelpath, artifact_path="pytorch_models")

ValueError: No configurable matching 'models.GRUmodel'.
  In file "test.gin", line 8
    models.GRUmodel.config = {